In [1]:
!pip install selenium
!pip install webdriver_manager

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()))

driver.get('https://www.etf.com/etfanalytics/etf-finder')

time.sleep(10)


In [ ]:
botao_100 = driver.find_element("xpath",
                               '/html/body/div[5]/section/div/div[3]/section/div/div/div/div/div[2]/section[2]/div[2]/section[2]/div[1]/div/div[4]/button')


driver.execute_script("arguments[0].click();", botao_100)

# quando o .click nn funcionar -> driver.execute_script("arguments[0].click();", botao_100)



In [ ]:
numero_paginas = driver.find_element("xpath", '/html/body/div[5]/section/div/div[3]/section/div/div/div/div/div[2]/section[2]/div[2]/section[2]/div[2]/div/label[2]')

numero_paginas = int(numero_paginas.text.replace("of ", ""))



In [ ]:
lista_tabela_por_pagina = []

elemento = driver.find_element("xpath", '//*[@id="finderTable"]')

botao_avancar_pagina = driver.find_element("xpath", '//*[@id="nextPage"]')


for pagina in range(1, numero_paginas + 1):
    
    html_tabela = elemento.get_attribute('outerHTML')
    
    tabela = pd.read_html(str(html_tabela))[0]
    
    lista_tabela_por_pagina.append(tabela)
        
    driver.execute_script("arguments[0].click();", botao_avancar_pagina)
    
    
tabela_cadastro_etfs = pd.concat(lista_tabela_por_pagina)

tabela_cadastro_etfs

In [ ]:
formulario_de_voltar_pagina = driver.find_element("xpath", '//*[@id="goToPage"]')

formulario_de_voltar_pagina.clear()
formulario_de_voltar_pagina.send_keys("1")
formulario_de_voltar_pagina.send_keys(u'\ue007')

In [ ]:
tab_performance = driver.find_element("xpath", "/html/body/div[5]/section/div/div[3]/section/div/div/div/div/div[2]/section[2]/div[2]/ul/li[2]/span")


tab_performance.click()


lista_tabela_por_pagina = []

elemento = driver.find_element("xpath", '//*[@id="finderTable"]')



for pagina in range(1, numero_paginas + 1):
    
    html_tabela = elemento.get_attribute('outerHTML')
    
    tabela = pd.read_html(str(html_tabela))[0]
    
    lista_tabela_por_pagina.append(tabela)
    
    driver.execute_script("arguments[0].click();", botao_avancar_pagina)
    
    
tabela_performance_etfs = pd.concat(lista_tabela_por_pagina)

tabela_performance_etfs


In [ ]:
driver.quit()

In [ ]:
tabela_performance_etfs = tabela_performance_etfs.set_index("Ticker")
tabela_performance_etfs = tabela_performance_etfs[['1 Year', '3 Years', '5 Years']]
tabela_cadastro_etfs = tabela_cadastro_etfs.set_index("Ticker")

base_de_dados_final = tabela_cadastro_etfs.join(tabela_performance_etfs, how = 'inner')

base_de_dados_final.to_csv('out.csv', encoding='utf-8')